"""classical_models.ipynb
by: Archie Gertsman (arkadiy2@illinois.edu)
Project director: Richard Sowers
r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/
Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license
"""

In [1]:
import sys
sys.path.append('../src/')
sys.path.append('../data/')
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from modeling_helpers import *

In [2]:
df = pd.read_pickle('../data/block4_updated.pkl')

agg_dict={
    'xtrack_dist': ['mean','std'],
    'avg_surr_speed': ['mean','std'],
#     'lanes':['mean'],
#     'len':['mean'],
    'speed':['mean','std'],
    'lon_acc': ['mean','std'],
    'lat_acc': ['mean','std']
}

df = df[list(agg_dict.keys()) + ['type']]
df.index = [df.index.map(lambda idx: f'{idx[0]}_{idx[1]}'),  
            df.index.get_level_values(2)]
df.index.names = ['id','road']

df = df.groupby(['id','road']).filter(lambda grp: len(grp) >= 50)
df

xtrack_dist  avg_surr_speed    speed  lon_acc  \
id       road                                                                  
4_1_1    250699362_250699984     -1.883401       10.464171  11.9046  -0.1145   
         250699362_250699984     -1.980795       10.457843  11.8975  -0.1007   
         250699362_250699984     -1.937041       10.452857  11.8919  -0.0918   
         250699362_250699984     -1.893287       10.448586  11.8871  -0.0869   
         250699362_250699984     -1.941984       10.444986  11.8831  -0.0784   
...                                    ...             ...      ...      ...   
1_3_2116 300400248_8446047162     0.658316       26.005850  25.3482  -0.3011   
         300400248_8446047162     0.764478       26.008150  25.3217  -0.3032   
         300400248_8446047162     0.736857       26.010500  25.2946  -0.3012   
         300400248_8446047162     0.783032       26.012950  25.2667  -0.3054   
         300400248_8446047162     0.755411       26.015600  25.2370  -0.3135   

                               lat_acc  type  
id       road                                 
4_1_1    250699362_250699984    0.0138  Taxi  
         250699362_250699984    0.0147  Taxi  
         250699362_250699984    0.0157  Taxi  
         250699362_250699984    0.0167  Taxi  
         250699362_250699984    0.0176  Taxi  
...                                ...   ...  
1_3_2116 300400248_8446047162   0.3234   Car  
         300400248_8446047162   0.3216   Car  
         300400248_8446047162   0.3180   Car  
         300400248_8446047162   0.3124   Car  
         300400248_8446047162   0.3045   Car  

[2992893 rows x 6 columns]

In [3]:
df_agg = downsample(df, window=50, overlap=0.3, min_speed_ratio=0.75, agg_dict=agg_dict)

In [4]:
print('car vs. taxi:')
accs_mean,accs_std = workflow(
    df_agg        = df_agg, 
    model         = Pipeline([('scaler', StandardScaler()), ('gbm', GradientBoostingClassifier())]), 
    splitter_obj  = StratifiedKFold(shuffle=True), 
    metric        = confusion_matrix, 
    balance_train = 'overall',
    balance_test  = True)
accs_mean,accs_std

car vs. taxi:
[[88 59]
 [61 86]]
[[91 56]
 [62 85]]
[[103  45]
 [ 71  77]]
[[86 62]
 [56 92]]
[[94 53]
 [74 73]]


(array([[92.4, 55. ],
        [64.8, 82.6]]),
 array([[5.95315043, 5.83095189],
        [6.67532771, 6.77052435]]))

In [5]:
acc_car = accs_mean[0,0] / accs_mean[0].sum()
acc_taxi = accs_mean[1,1] / accs_mean[1].sum()
overall = [acc_car, acc_taxi]
overall

[0.6268656716417911, 0.5603799185888738]